In [1]:
# Assignment_2: Akhilesh Ravi-16110011
#Using Jane Austen Novels for Language Model Generation

# Importing all packages
import math
import numpy as np
from operator import itemgetter
from nltk import word_tokenize, sent_tokenize
from string import punctuation
from nltk.util import ngrams

In [2]:
import re
from sklearn.model_selection import train_test_split 

# Pre-Processing of the dataset

DATASET- Jane Austen Novels: The Complete Works of Jane Austen take one line, neglect empty line and then add it to line_set then lower down whole set(string)


In [5]:
with open("JaneAusten.txt", 'r') as file:  
    lines = ['']
    for line in (l.rstrip() for l in file):
        if line != '' or lines[-1] != '\n':                 
            lines.append(line + '\n')
    text = "".join(lines)
text = text.lower()  

# Tokenizing sentences
# Vocabulary dictionary and corpus
vocabulary = {}
corpus = []
list_sentence = sent_tokenize(text)
# <s> is added to the start of each sentence
# </s> is added to the end of sentences
for sentence in list_sentence:
    word_list = word_tokenize(sentence) # tokenizing each word of that sentence
    final_sentence = ['<s>']
    for word in word_list:
        # Obtains the frequency of the words in the corpus
        if len(set(word).intersection(punctuation)) == 0 and (len(word)>1 or word == 'a' or word == 'i') :
            final_sentence.append(word)
            if word not in vocabulary.keys(): 
                vocabulary[word] = 1
            else:
                vocabulary[word] += 1
                
    final_sentence += ['</s>']
    corpus += [" ".join(final_sentence)] #adding processed sentence to the corpus

#Now Processed Corpus is ready    
number_of_sentence = len(corpus)
# Vocabulary list doesn't contain the START and END word used in Pre-Processing
vocabulary['<s>'] = 2*number_of_sentence
vocabulary['</s>'] = 2*number_of_sentence
# Now we have Processed sentence corpus and vocabulary list. 
#Processed Corpus DATA ANALYSIS  
tokens = sum(vocabulary.values())
types = len(vocabulary)
print ('Number of Sentences = ' + str(number_of_sentence))
print ('Number of Types = ' + str(types))
print ('Number of Tokens = ' + str(tokens))
print ("TTR= " + str(types/tokens))
# train-test split in 80:20 ratio
train_data, test_data = train_test_split(corpus, test_size=0.20, random_state=64)   
    

Number of Sentences = 35056
Number of Types = 14651
Number of Tokens = 914639
TTR= 0.01601834166266691


In [6]:
corpus[:12]

['<s> project gutenberg the complete works of jane austen by jane austen this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever </s>',
 '<s> you may copy it give it away or it under the terms of the project gutenberg license included with this ebook or online at title the complete project gutenberg works of jane austen author jane austen editor david widger release date january 25 2010 ebook 31100 language english character set encoding ascii start of this project gutenberg ebook the works of jane austen produced by many project gutenberg volunteers </s>',
 '<s> the works of jane austen edited by david widger project gutenberg editions dedication this jane austen collection is dedicated to alice goodson hart woodby note the accompanying html file has active links to all the volumes and chapters in this set </s>',
 '<s> contents persuasion northanger abbey mansfield park emma lady susan love and freindship and other early works pride and preju

# Q1a MLE for N-Grams : N=  1,2,3,4

In [9]:
# Unigrams
MLE_u = {}           
for i in vocabulary.keys():
    MLE_u[i] = (float(vocabulary[i]))/tokens     


# Bigrams

MLE_b = {}# initilaizing empty biagram MLE
bigram_list = {} # initilaizing empty biagram list
# from each sentence in corpus extracting all the biagrams
for sentence in corpus:
    words_in_sentence = sentence.split()
    for i in range(1,len(words_in_sentence)):
        j = (words_in_sentence[i-1], words_in_sentence[i])#taking all consecutive combination
        try:
            bigram_list[j] += 1
        except:
            bigram_list[j] = 1
#NOw we have list a histogram of biagram in the corpus             
for i in bigram_list.keys():
    MLE_b[i] = float(bigram_list[i])/vocabulary[i[0]]


# Trigrams
MLE_t = {}#trigram MLE 
trigram_list = {}# initializing the list

for sentence in corpus:
    words_in_sentence = sentence.split()
    for i in range(2,len(words_in_sentence)):
        j = (words_in_sentence[i-2], words_in_sentence[i-1], words_in_sentence[i])
        try:
            trigram_list[j] += 1
        except:
            trigram_list[j] = 1

#NOw we have list a histogram of trigram in the corpus             
for i in trigram_list.keys():
    MLE_t[i] = (trigram_list[i]*1.0)/bigram_list[(i[0], i[1])]

# Quadrigrams
MLE_q = {}
quadgram_list = {}

for sentence in corpus:
    words_in_sentence = sentence.split()
    for i in range(3,len(words_in_sentence)):
        j = (words_in_sentence[i-3], words_in_sentence[i-2], words_in_sentence[i-1], words_in_sentence[i])
        try:
            quadgram_list[j] += 1
        except:
            quadgram_list[j] = 1

#NOw we have list a histogram of QUADgram in the corpus             
for i in quadgram_list.keys():
    MLE_q[i] = (quadgram_list[i]*1.0)/trigram_list[(i[0], i[1], i[2])]

In [14]:
print("UNIGRAM-MLE - First 10\n",sorted(MLE_u.items(), key=lambda x:x[1])[:10],
"\nBIGRAM-MLE - First 10\n",sorted(MLE_b.items(), key=lambda x:x[1])[:10],
"\nTRIGRAM-MLE - First 10\n",sorted(MLE_t.items(), key=lambda x:x[1])[:10],
"\nQUADGRAM-MLE - First 10\n",sorted(MLE_q.items(), key=lambda x:x[1])[:10])

UNIGRAM-MLE - First 10
 [('2010', 1.0933275314085666e-06), ('31100', 1.0933275314085666e-06), ('encoding', 1.0933275314085666e-06), ('edited', 1.0933275314085666e-06), ('dedication', 1.0933275314085666e-06), ('goodson', 1.0933275314085666e-06), ('woodby', 1.0933275314085666e-06), ('html', 1.0933275314085666e-06), ('patents', 1.0933275314085666e-06), ('1760', 1.0933275314085666e-06)] 
BIGRAM-MLE - First 10
 [(('<s>', 'precisely'), 1.4262893655864902e-05), (('<s>', 'herself'), 1.4262893655864902e-05), (('<s>', 'journeys'), 1.4262893655864902e-05), (('<s>', 'quit'), 1.4262893655864902e-05), (('<s>', 'kellynch'), 1.4262893655864902e-05), (('<s>', 'therefore'), 1.4262893655864902e-05), (('<s>', 'sailors'), 1.4262893655864902e-05), (('<s>', 'fellow'), 1.4262893655864902e-05), (('<s>', 'picture'), 1.4262893655864902e-05), (('<s>', 'penelope'), 1.4262893655864902e-05)] 
TRIGRAM-MLE - First 10
 [(('<s>', 'i', 'venture'), 0.0002638522427440633), (('<s>', 'i', 'meet'), 0.0002638522427440633), (('

In [15]:
#TASK 1a . HOW MANY N-GRAMS are POSSIBLE AND HOW MANY ACTUALLY EXISTS
def NGRAMS(N):
    possible = types**N
    list_of_NGRAMS = []
    for sentence in corpus:
        list_of_NGRAMS += list(ngrams(sentence.split(), N))
    set_of_NGRAMS = set(list_of_NGRAMS)
    present = len(set_of_NGRAMS)
    return possible, present

ngram = input('Enter value of N:')
ngram=int(ngram)
possible, present = NGRAMS(ngram)
print ("Number of possible n-grams: " + str(possible))
print ("Number of present n-grams: " + str(present))

Enter value of N:2
Number of possible n-grams: 214651801
Number of present n-grams: 213027


In [ ]:
#TASK 1b - Defining Generator(model_name) for sentence generation 

# Using Inbuilt Sampling Function from  the Multinomial 
# GENRATOR FUNCTION DEFINATION

def Generator(model_name):
    if model_name=="UNIGRAM":
        sentence = ''
        start = "<s>"
        sentence = sentence + start
        probability = list(MLE_u.values())
        wordlist = list(MLE_u.keys())
        while(True):
            arr = np.random.multinomial(1,probability)
            word = wordlist[int(np.where(arr == 1)[0])]
            if (word=="<s>"):
                continue
            elif (word == "</s>"):
                sentence=sentence + word
                print (sentence)
                return
            else:
                sentence=sentence+word+" "
                
    elif model_name=="BIGRAM":
        sentence="<s>"
        start = "<s>"
        while(True):
            nextw=[]
            p=[]
            for i in MLE_b.keys():
                if(i[0]==start):
                    nextw.append(i[1])
            arr = (np.random.multinomial(1.000000000000000000000,p))
            w = nextw[int(np.where(arr == 1)[0])]
            if w == "</s>":
                sentence=sentence+w
                print (sentence)
                return
            else:
                sentence=sentence+w+" "
                start=w
                
    elif model_name=="TRIGRAM":
        sentence = " <s> "
        start = "<s>"
        nextw=[]
        p=[]
        for i in MLE_b.keys():
            if(i[0]==start):
                nextw.append(i[1])
                p.append(MLE_b[i])
            
        arr = np.random.multinomial(1,p)
        w = nextw[int(np.where(arr==1)[0])]
        bigTuple = (start,w)
        sentence=sentence+w+" "
            
        while(True):
            nextwT=[]
            pT=[]
            for i in MLE_t.keys():
                if(i[0]==bigTuple[0] and i[1] == bigTuple[1]):
                    nextwT.append(i[2])
                    pT.append(MLE_t[i])
            
            arrT = np.random.multinomial(1,pT)
            wT = nextwT[int(np.where(arrT==1)[0])]
            bigTuple = (bigTuple[1],wT)
            
            
            
            if wT == "</s>":
                sentence=sentence+wT
                print (sentence)
                return
            else:
                sentence=sentence+wT+" "
                
                
    elif model_name == "QUADGRAM":
        sentence = "<s> "
        start = "<s>"
        nextwB=[]
        pB=[]
        for i in MLE_b.keys():
            if(i[0]==start):
                nextwB.append(i[1])
                pB.append(MLE_b[i])
            
        arrB = np.random.multinomial(1,pB)
        wB = nextwB[int(np.where(arrB==1)[0])]
        bigTuple = (start,wB)
        sentence=sentence+wB+" "
            
        nextwT=[]
        pT=[]
        for i in MLE_t.keys():
            if(i[0] == bigTuple[0] and i[1]== bigTuple[1]):
                nextwT.append(i[2])
                pT.append(MLE_t[i])
        
        arrT = np.random.multinomial(1,pT)
        wT = nextwT[int(np.where(arrT==1)[0])]
        trigTuple = (start,wB,wT)
        sentence=sentence+wT+" "
            
        while(True):
            nextwQ=[]
            pQ=[]
            for i in MLE_q.keys():
                if(i[0]==trigTuple[0] and i[1] == trigTuple[1] and i[2]==trigTuple[2]):
                    nextwQ.append(i[3])
                    pQ.append(MLE_q[i])
            
            arrQ = np.random.multinomial(1,pQ)
            wQ = nextwQ[int(np.where(arrQ==1)[0])]
            trigTuple = (trigTuple[1],trigTuple[2],wQ)       


        if wQ == "</s>":
            sentence=sentence+wQ
            print (sentence)
            return
        else:
            sentence=sentence+wQ+" "  

print ("----------------------------UNIGRAM SENTENCES----------------------------------")
for z in range(5):
    Generator("UNIGRAM")  
    
print ("----------------------------BIGRAM SENTENCES-------------------------------------")
for z in range(5):
    Generator("BIGRAM")

print ("-----------------------------TRIGRAM SENTENCES----------------------------------")
for z in range(5):
    Generator("TRIGRAM")

print ("------------------------------QUADGRAM SENTENCES--------------------------------")
for z in range(5):
    Generator("QUADGRAM")

In [132]:
def sample_multinomial(w):
    p = softmax(w)
    x = np.random.uniform(0,1)
    for i,v in enumerate(np.cumsum(p)):
        if x < v: return i
    return len(p)-1 

def generatorSent(modelName):
    if modelName=="unigram":
        s = ''
        start = "<s>"
        s = s + start
        prob = list(unigramMLE.values())
        wordlist = list(unigramMLE.keys())
        while(True):
            arr = np.random.multinomial(1,prob)
            word = wordlist[int(np.where(arr == 1)[0])]
            if (word=="<s>"):
                continue
            elif (word == "</s>"):
                s=s + word
                print (s)
                return
            else:
                s=s+word+" "
                
    elif modelName=="bigram":
        s="<s>"
        start = "<s>"
        while(True):
            nextw=[]
            p=[]
            for i in bigramMLE.keys():
                if(i[0]==start):
                    nextw.append(i[1])
                    p.append(bigramMLE[i])     # If you get an error ValueError: sum(pvals[:-1]) > 1.0 then it is because precision of the numbers 
                                                # in Python are not exact please Change your system to 32-bit and run it again. Or run this part by part
            arr = (np.random.multinomial(1.000000000000000000000,p))
            w = nextw[int(np.where(arr == 1)[0])]
            if w == "</s>":
                s=s+w
                print (s)
                return
            else:
                s=s+w+" "
                start=w
                
    elif modelName=="trigram":
        s = " <s> "
        start = "<s>"
        nextw=[]
        p=[]
        for i in bigramMLE.keys():
            if(i[0]==start):
                nextw.append(i[1])
                p.append(bigramMLE[i])
            
        arr = np.random.multinomial(1,p)
        w = nextw[int(np.where(arr==1)[0])]
        bigTuple = (start,w)
        s=s+w+" "
            
        while(True):
            nextwT=[]
            pT=[]
            for i in trigramMLE.keys():
                if(i[0]==bigTuple[0] and i[1] == bigTuple[1]):
                    nextwT.append(i[2])
                    pT.append(trigramMLE[i])
            
            arrT = np.random.multinomial(1,pT)
            wT = nextwT[int(np.where(arrT==1)[0])]
            bigTuple = (bigTuple[1],wT)
            
            
            
            if wT == "</s>":
                s=s+wT
                print (s)
                return
            else:
                s=s+wT+" "
                
                
    elif modelName == "quadgram":
        s = "<s> "
        start = "<s>"
        nextwB=[]
        pB=[]
        for i in MLE_b.keys():
            if(i[0]==start):
                nextwB.append(i[1])
                pB.append(MLE_b[i])
            
        arrB = np.random.multinomial(1,pB)
        wB = nextwB[int(np.where(arrB==1)[0])]
        bigTuple = (start,wB)
        s=s+wB+" "
            
        nextwT=[]
        pT=[]
        for i in MLE_t.keys():
            if(i[0] == bigTuple[0] and i[1]== bigTuple[1]):
                nextwT.append(i[2])
                pT.append(MLE_t[i])
        
        arrT = np.random.multinomial(1,pT)
        wT = nextwT[int(np.where(arrT==1)[0])]
        trigTuple = (start,wB,wT)
        s=s+wT+" "
            
        while(True):
            nextwQ=[]
            pQ=[]
            for i in MLE_q.keys():
                if(i[0]==trigTuple[0] and i[1] == trigTuple[1] and i[2]==trigTuple[2]):
                    nextwQ.append(i[3])
                    pQ.append(MLE_q[i])
            
            arrQ = np.random.multinomial(1,pQ)
            wQ = nextwQ[int(np.where(arrQ==1)[0])]
            trigTuple = (trigTuple[1],trigTuple[2],wQ)       
            
            
            if wQ == "</s>":
                s=s+wQ
                print (s)
                return
            else:
                s=s+wQ+" "      

print ("Unigram Sentences are")
print ("-------------------------")
for z in range(5):
    generatorSent("unigram")  
    
print
print
print ("Bigrams Sentences are")
print ("-------------------------")
for z in range(5):
    generatorSent("bigram")

print
print
print ("Trigrams Sentences are")
print ("-------------------------")
for z in range(5):
    generatorSent("trigram")

print
print
print ("Quadgrams Sentences are")
print ("-------------------------")
for z in range(5):
    generatorSent("quadgram")

Unigram Sentences are
-------------------------
<s>daughter she my you that they but </s>
<s>speak man to </s>
<s>and was chilly and so stay still she unfeeling compassion said not emma mary but everything in </s>
<s>conceit the had about which i of sleek ashamed he father unnecessary like a have late world it nor camden take before wonder can example sure quite judge here body that one you what it from said act was spirits it be the someone henry not </s>
<s>fears bed there my darcy should more as but not the the </s>
Bigrams Sentences are
-------------------------
<s>professor michael hart woodby note was spent scarcely anything about she could but alas i was she grew impatient to her marriage explained them too much the more of the day following monday trusting their being a letter did not come cried isabella is i did not long very thoroughly fagged fanny shocked but what was not belong to avoid </s>
<s>professor michael hart and warmth which was now on the course both sides there y

TypeError: only length-1 arrays can be converted to Python scalars

In [ ]:
# Classical Approach and this code is for getting N-grams 
# we will pass the processed data to the generator and it will generate the n-grams where n=parameter
def GENERATE_NGRAMS(text, N):
    
    # Replace all none alphanumeric characters with spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    
    # Breaking sentence in the token, removing empty tokens- further <S> and </S> are tokens
    tokens = [token for token in text.split(" ") if token != ""]
    
    # Concatenating the tokens into NGRAMS and returning using ZIP function
    NGRAMS = zip(*[token[i:] for i in range(N)])
    return [" ".join(NGRAM) for NGRAM in NGRAMS]




In [ ]:
#Classical Approach and this code make a GEnerator function


In [ ]:
# this code is for evaluation of Classiscal Approach


In [ ]:
#this code for Neural approach 


In [ ]:
#this code evaluate the Neural Approach
